In [1]:
"""
probabilistic_lstm_forecast.py

Advanced time series forecasting with an LSTM producing probabilistic outputs (mean and log-variance).
Includes:
 - synthetic daily dataset generation (trend + seasonality + heteroscedastic noise)
 - feature engineering (cyclic time features)
 - sequence generation for supervised learning
 - LSTM model that outputs mu and log_var and is trained with Gaussian NLL
 - Monte Carlo Dropout for epistemic uncertainty estimation
 - Evaluation: RMSE, MAE, coverage and mean interval width (80% and 95%)
 - Visualization of forecasts vs actuals with prediction intervals

Author: ChatGPT (GPT-5 Thinking mini)
Date: 2025-11
"""

import os
from typing import Tuple, Dict, List

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Ensure reproducibility
SEED = 42
np.random.seed(SEED)

# If running in an environment with TensorFlow:
try:
    import tensorflow as tf
    from tensorflow.keras import layers, models, backend as K
    tf.random.set_seed(SEED)
    TF_AVAILABLE = True
except Exception:
    TF_AVAILABLE = False

# -------------------------
# Utility functions
# -------------------------


def generate_synthetic_daily_series(
    start: str = "2018-01-01",
    days: int = 3 * 365,
    trend_slope: float = 0.001,
) -> pd.DataFrame:
    """
    Generate synthetic daily time series with trend, yearly and weekly seasonality,
    and heteroscedastic noise.

    Parameters
    ----------
    start : str
        Start date for the series.
    days : int
        Number of days to generate.
    trend_slope : float
        Linear trend per day.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns ['ds', 'y'] where ds is datetime and y is value.
    """
    rng = pd.date_range(start=start, periods=days, freq="D")
    t = np.arange(days).astype(float)

    trend = trend_slope * t
    yearly = 2.0 * np.sin(2 * np.pi * t / 365.25)
    weekly = 0.5 * np.sin(2 * np.pi * t / 7.0)

    base_noise_scale = 0.5 + 0.5 * (1 + np.sin(2 * np.pi * t / 365.25))
    noise = np.random.normal(scale=base_noise_scale)
    outliers = np.random.choice([0.0, 0.0, 0.0, 3.0], size=days, p=[0.7, 0.15, 0.14, 0.01])
    y = 10 + trend + yearly + weekly + noise + outliers

    df = pd.DataFrame({"ds": rng, "y": y})
    return df


def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create time-based features (cyclic encodings) for the time series.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame with 'ds' and 'y'.

    Returns
    -------
    pd.DataFrame
        DataFrame with additional features.
    """
    df = df.copy()
    df["day_of_year"] = df["ds"].dt.dayofyear
    df["day_of_week"] = df["ds"].dt.dayofweek
    df["month"] = df["ds"].dt.month
    df["sin_doy"] = np.sin(2 * np.pi * df["day_of_year"] / 365.25)
    df["cos_doy"] = np.cos(2 * np.pi * df["day_of_year"] / 365.25)
    df["sin_dow"] = np.sin(2 * np.pi * df["day_of_week"] / 7.0)
    df["cos_dow"] = np.cos(2 * np.pi * df["day_of_week"] / 7.0)
    return df


def build_sequence_dataset(
    df: pd.DataFrame,
    feature_cols: List[str],
    target_col: str,
    lookback: int,
    horizon: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Create sequences (X) and targets (Y) for supervised learning.

    Returns X shape (samples, lookback, features), Y shape (samples, horizon).
    """
    values = df[feature_cols + [target_col]].values
    n_features = len(feature_cols)
    X, Y = [], []
    for i in range(len(df) - lookback - horizon + 1):
        x = values[i : i + lookback, :n_features]
        y = values[i + lookback : i + lookback + horizon, -1]
        X.append(x)
        Y.append(y)
    return np.array(X), np.array(Y)


# -------------------------
# Probabilistic loss and model (TensorFlow implementation)
# -------------------------


def gaussian_nll(y_true, y_pred):
    """
    Gaussian negative log-likelihood loss. Expects y_pred to contain [mu, log_var].
    """
    mu = y_pred[:, 0:1]
    log_var = y_pred[:, 1:2]
    precision = K.exp(-log_var)
    return K.mean(0.5 * precision * K.square(y_true - mu) + 0.5 * log_var)


def build_lstm_probabilistic(input_shape: Tuple[int, int], units: int = 64, dropout_rate: float = 0.2):
    """
    Build an LSTM model that outputs mean and log-variance (aleatoric).
    Dropout layers use `training=True` at call-time so MC Dropout is possible.
    """
    inp = layers.Input(shape=input_shape)
    x = layers.LSTM(units, return_sequences=False)(inp)
    # Dropout with training=True in inference: set training argument when calling model
    x = layers.Dropout(dropout_rate)(x, training=True)
    x = layers.Dense(units // 2, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x, training=True)
    mu = layers.Dense(1, name="mu")(x)
    log_var = layers.Dense(1, name="log_var")(x)
    out = layers.Concatenate(name="out")([mu, log_var])
    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=gaussian_nll)
    return model


def mc_dropout_predict(model, X: np.ndarray, mc_samples: int = 100):
    """
    Perform MC Dropout predictions to estimate predictive mean, epistemic var, and aleatoric var.

    Returns:
        mu_mean (n,), epistemic_var (n,), aleatoric_mean (n,)
    """
    preds = []
    for _ in range(mc_samples):
        pred = model(X, training=True)
        preds.append(pred.numpy())
    preds = np.stack(preds, axis=0)  # shape (mc, batch, 2)
    mu_samples = preds[:, :, 0]
    log_var_samples = preds[:, :, 1]
    mu_mean = mu_samples.mean(axis=0)
    epistemic_var = mu_samples.var(axis=0)
    aleatoric_mean = np.exp(log_var_samples).mean(axis=0)
    return mu_mean.ravel(), epistemic_var.ravel(), aleatoric_mean.ravel()


# -------------------------
# Evaluation helper
# -------------------------


def evaluate_intervals(y_true, mu, total_var, ci_list=(0.8, 0.95)):
    """
    Compute RMSE, MAE and coverage/mean interval width for given CIs.
    """
    metrics = {}
    rmse = np.sqrt(mean_squared_error(y_true, mu))
    mae = mean_absolute_error(y_true, mu)
    metrics["rmse"] = float(rmse)
    metrics["mae"] = float(mae)
    for ci in ci_list:
        if np.isclose(ci, 0.8):
            z = 1.281551565545
        elif np.isclose(ci, 0.9):
            z = 1.6448536269514722
        elif np.isclose(ci, 0.95):
            z = 1.959963984540054
        else:
            # fallback
            from scipy.stats import norm  # scipy is common in ML envs
            z = float(norm.ppf(0.5 + ci / 2.0))
        half_width = z * np.sqrt(total_var)
        lower = mu - half_width
        upper = mu + half_width
        coverage = float(np.mean((y_true >= lower) & (y_true <= upper)))
        mean_width = float(np.mean(upper - lower))
        metrics[f"coverage_{int(ci*100)}"] = coverage
        metrics[f"mean_width_{int(ci*100)}"] = mean_width
    return metrics


# -------------------------
# Main pipeline function
# -------------------------


def run_pipeline(
    days: int = 3 * 365,
    lookback: int = 30,
    horizon: int = 1,
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
    epochs: int = 20,
    batch_size: int = 32,
    units: int = 64,
    dropout_rate: float = 0.2,
    mc_samples: int = 100,
    artifact_dir: str = "ts_forecast_outputs",
):
    """
    Run the full pipeline: data generation, preprocessing, training, prediction, evaluation, plotting.

    Returns a dictionary of metrics and the artifact directory.
    """
    if not TF_AVAILABLE:
        raise RuntimeError("TensorFlow is not available in this environment. Install tensorflow and retry.")

    # 1) Data
    df = generate_synthetic_daily_series(days=days)
    df = create_features(df)
    feature_cols = ["sin_doy", "cos_doy", "sin_dow", "cos_dow"]
    X_all, Y_all = build_sequence_dataset(df, feature_cols, "y", lookback, horizon)
    Y_all = Y_all.ravel()

    n_samples = X_all.shape[0]
    train_end = int(n_samples * train_ratio)
    val_end = int(n_samples * (train_ratio + val_ratio))

    X_train, Y_train = X_all[:train_end], Y_all[:train_end]
    X_val, Y_val = X_all[train_end:val_end], Y_all[train_end:val_end]
    X_test, Y_test = X_all[val_end:], Y_all[val_end:]

    n_features = X_train.shape[2]
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    scaler_X.fit(X_train.reshape(-1, n_features))
    X_train_scaled = scaler_X.transform(X_train.reshape(-1, n_features)).reshape(X_train.shape)
    X_val_scaled = scaler_X.transform(X_val.reshape(-1, n_features)).reshape(X_val.shape)
    X_test_scaled = scaler_X.transform(X_test.reshape(-1, n_features)).reshape(X_test.shape)

    scaler_y.fit(Y_train.reshape(-1, 1))
    Y_train_scaled = scaler_y.transform(Y_train.reshape(-1, 1)).ravel()
    Y_val_scaled = scaler_y.transform(Y_val.reshape(-1, 1)).ravel()
    Y_test_scaled = scaler_y.transform(Y_test.reshape(-1, 1)).ravel()

    # 2) Model
    input_shape = (lookback, n_features)
    model = build_lstm_probabilistic(input_shape, units=units, dropout_rate=dropout_rate)
    model.summary()

    # For NLL training, we need two outputs per sample: mu and log_var.
    # For a stable initialization, set log_var to small positive constant for targets in y-scale.
    init_log_var = np.log(0.1)
    y_train_targets = np.vstack([Y_train_scaled, np.ones_like(Y_train_scaled) * init_log_var]).T
    y_val_targets = np.vstack([Y_val_scaled, np.ones_like(Y_val_scaled) * init_log_var]).T

    # 3) Training with early stopping
    early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
    history = model.fit(
        X_train_scaled,
        y_train_targets,
        validation_data=(X_val_scaled, y_val_targets),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early],
        verbose=2,
    )

    # 4) Prediction with MC Dropout
    mu_pred_scaled, epistemic_var_scaled, aleatoric_mean_scaled = mc_dropout_predict(
        model, X_test_scaled, mc_samples=mc_samples
    )

    # Convert predicted means / variances back to original scale
    mu_pred = scaler_y.inverse_transform(mu_pred_scaled.reshape(-1, 1)).ravel()
    y_std = scaler_y.scale_[0]
    aleatoric_mean = aleatoric_mean_scaled * (y_std ** 2)  # model predicted variance in scaled units -> original
    epistemic_var = epistemic_var_scaled * (y_std ** 2)
    total_var = aleatoric_mean + epistemic_var

    # 5) Evaluate
    metrics = evaluate_intervals(Y_test, mu_pred, total_var, ci_list=(0.8, 0.95))

    # 6) Save artifacts and plots
    os.makedirs(artifact_dir, exist_ok=True)
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv(os.path.join(artifact_dir, "metrics.csv"), index=False)

    # Plot several segments from test set
    n_plots = 6
    step = max(1, len(Y_test) // n_plots)
    plot_indices = list(range(0, len(Y_test), step))[:n_plots]
    for idx in plot_indices:
        start = max(0, idx - 60)
        end = min(len(Y_test), idx + 60)
        mu_seg = mu_pred[start:end]
        y_seg = Y_test[start:end]
        total_var_seg = total_var[start:end]
        days_idx = np.arange(start, end)

        z95 = 1.959963984540054
        half_width95 = z95 * np.sqrt(total_var_seg)
        lower95 = mu_seg - half_width95
        upper95 = mu_seg + half_width95

        z80 = 1.281551565545
        half_width80 = z80 * np.sqrt(total_var_seg)
        lower80 = mu_seg - half_width80
        upper80 = mu_seg + half_width80

        plt.figure(figsize=(10, 5))
        plt.plot(days_idx, y_seg, label="Actual")
        plt.plot(days_idx, mu_seg, label="Forecast (mean)")
        plt.fill_between(days_idx, lower95, upper95, alpha=0.2, label="95% PI")
        plt.fill_between(days_idx, lower80, upper80, alpha=0.4, label="80% PI")
        plt.legend()
        plt.title(f"Forecast vs Actual (indices {start}:{end})")
        fname = os.path.join(artifact_dir, f"forecast_plot_{start}_{end}.png")
        plt.savefig(fname)
        plt.close()

    return {"metrics": metrics, "artifact_dir": artifact_dir, "history": history.history}


# -------------------------
# If running as script
# -------------------------
if __name__ == "__main__":
    # Quick-run configuration — adjust as needed for longer training/tuning
    run_config = dict(
        days=3 * 365,
        lookback=30,
        horizon=1,
        train_ratio=0.7,
        val_ratio=0.15,
        epochs=15,
        batch_size=32,
        units=64,
        dropout_rate=0.2,
        mc_samples=100,
        artifact_dir="ts_forecast_outputs",
    )

    if not TF_AVAILABLE:
        print("TensorFlow not installed in this environment. To run this script install TensorFlow:")
        print("  pip install 'tensorflow>=2.11'  # or use tensorflow-cpu if you prefer")
        raise SystemExit(1)

    results = run_pipeline(**run_config)
    print("Results metrics:")
    for k, v in results["metrics"].items():
        print(f"  {k}: {v:.6f}")
    print("Artifacts saved in:", results["artifact_dir"])


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 4)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 64)                │          17,664 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 64)                │               0 │ lstm[0][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 32)                │           2,080 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 32)                │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ mu (Dense)                    │ (None, 1)                 │              33 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ log_var (Dense)               │ (None, 1)                 │              33 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ out (Concatenate)             │ (None, 2)                 │               0 │ mu[0][0], log_var[0][0]    │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 19,810 (77.38 KB)

 Trainable params: 19,810 (77.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
24/24 - 4s - 187ms/step - loss: 1.0872 - val_loss: 1.1818
Epoch 2/15
24/24 - 0s - 19ms/step - loss: 0.7674 - val_loss: 1.0833
Epoch 3/15
24/24 - 0s - 20ms/step - loss: 0.7199 - val_loss: 1.0814
Epoch 4/15
24/24 - 0s - 19ms/step - loss: 0.7280 - val_loss: 1.0856
Epoch 5/15
24/24 - 0s - 19ms/step - loss: 0.7157 - val_loss: 1.0844
Epoch 6/15
24/24 - 0s - 19ms/step - loss: 0.6988 - val_loss: 1.0789
Epoch 7/15
24/24 - 0s - 19ms/step - loss: 0.6982 - val_loss: 1.0753
Epoch 8/15
24/24 - 0s - 19ms/step - loss: 0.6961 - val_loss: 1.0742
Epoch 9/15
24/24 - 0s - 19ms/step - loss: 0.6911 - val_loss: 1.0858
Epoch 10/15
24/24 - 0s - 19ms/step - loss: 0.6963 - val_loss: 1.0803
Epoch 11/15
24/24 - 0s - 19ms/step - loss: 0.6885 - val_loss: 1.0757
Epoch 12/15
24/24 - 0s - 19ms/step - loss: 0.6859 - val_loss: 1.0809
Results metrics:
  rmse: 1.764058
  mae: 1.584527
  coverage_80: 0.812500
  mean_width_80: 4.258481
  coverage_95: 0.987500
  mean_width_95: 6.512785
Artifacts saved in: ts_forecas